## Here is panos code
We need to create 3 tests with brute force search(O(n))
1) with Linf metric
2) with Linf + Clustering
3) with Linf complete
4) with L2 metric

In [3]:
import numpy as np

In [13]:
# define Linf() function for a square and a point
def Linf(square, point):
    # 1st put coords around point and transform
    # in order to do the transform substract the point from each coord
    square_new = np.array([np.subtract(square_point, point) for square_point in square])
    # find if y = abs(x) intersects the square and if so add the point to the list so that the clockwise order is preserved
    # find the intersection point
    # find if y = abs(x) intersects the square
    # for each edge of the square, check if y = abs(x) intersects the edge
    for i in range(square.shape[0]):
        sq_pt1 = square_new[i]
        sq_pt2 = square_new[(i+1)%square.shape[0]]
        # alpha 
        if not (sq_pt2[0] - sq_pt1[0] == 0):
            a = (sq_pt2[1] - sq_pt1[1])/(sq_pt2[0] - sq_pt1[0])
        else:
            continue
        # beta
        b = sq_pt1[1] - a*sq_pt1[0]
        # new point
        if not (a == 1 or a == -1):
            new_pts = np.array([[b/(a-1), b/(a-1)],[b/(a+1), -b/(a+1)]])
        else:
            continue
        # check if new points lie inside the segment
        # if so add to square_new
        for new_pt in new_pts:
            if (new_pt[0] >= min(sq_pt1[0], sq_pt2[0])) and (new_pt[0] <= max(sq_pt1[0], sq_pt2[0])) and \
                (new_pt[1] >= min(sq_pt1[1], sq_pt2[1])) and (new_pt[1] <= max(sq_pt1[1], sq_pt2[1])):
                square_new = np.insert(square_new, i+1, new_pt, axis=0)
    # get Linf from each point in square_new
    min_dist = np.inf
    for square_point in square_new:
        dist = np.max(np.abs(square_point))
        if dist < min_dist:
            min_dist = dist
    return min_dist


In [18]:
# define simplified Linf
def Linf_(square, q_point):
    min_dist = np.inf
    for point in square:
        dist = np.max(np.abs(np.subtract(point, q_point)))
        if dist < min_dist:
            min_dist = dist
    return min_dist

In [19]:
# define a square with 4 points
sq_pts = np.array([(0,0), (0,1), (1,1), (1,0)])
# define Linf from a square and a point
Linf_dist = Linf_(sq_pts, [3,3])
print("Linf distance from square to point is: ", Linf_dist)


Linf distance from square to point is:  2


In [4]:
np.subtract([1,2,3], [1,2,3])

array([0, 0, 0])

In [8]:
from utils import geometry as geo
import numpy as np

In [16]:
geo.IsPointInsidePoly([1,1], np.array([[0,0], [-1,1], [0,2], [1,1]]))

True

In [31]:
# Create 10000 querry points where each point is a 2D point and does not overlap the square
# first load the 100sq.npy file
sq_pts = np.load('./data_v1/1000sq.npy')
# create a list of points
q_pts = []
num_pts = 10000
while len(q_pts) < num_pts:
    if len(q_pts) % 1000 == 0:
        print("Number of points generated: ", len(q_pts))
    # generate a point from x = [3,297] and y = [3,297]
    x = np.random.randint(3, 297)
    y = np.random.randint(3, 297)
    # check if the point is inside the square
    label = False
    for sq_pt in sq_pts:
        if geo.IsPointInsidePoly([x,y], geo.create_square2(sq_pt)):
            label = True
    if not label:
        q_pts.append([x,y])

Number of points generated:  0
Number of points generated:  1000
Number of points generated:  2000
Number of points generated:  2000
Number of points generated:  3000
Number of points generated:  4000
Number of points generated:  5000
Number of points generated:  6000
Number of points generated:  7000
Number of points generated:  8000
Number of points generated:  9000


In [32]:
#save sq_pts
np.save('./data_v1/1000sq/1000qp.npy', np.array(q_pts))